In [8]:
import pandas as pd
import numpy as np

import fbprophet

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train['Course_ID'] = train['Course_ID'].astype('object')
train['Day_No'] = train['Day_No'].astype('object')

In [3]:
train.tail()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
512082,548023,878,600,Software Marketing,Program,0,0,1,8904,0.07,114
512083,548024,879,600,Software Marketing,Program,0,0,1,10542,0.07,145
512084,548025,880,600,Software Marketing,Program,0,0,1,13671,0.07,167
512085,548026,881,600,Software Marketing,Program,0,0,1,8904,0.07,107
512086,548027,882,600,Software Marketing,Program,1,0,1,11445,0.07,152


In [4]:
test.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric
0,883,883,1,Development,Course,1,0,1,0.007
1,884,884,1,Development,Course,1,0,1,0.007
2,885,885,1,Development,Course,1,0,1,0.007
3,886,886,1,Development,Course,1,0,1,0.007
4,887,887,1,Development,Course,0,0,1,0.007


In [5]:
# USing prophet to model user traffic

train_ = train[['Day_No','Course_ID','Sales']]
temp = pd.DataFrame(columns=['Day_No'])
n =test.Day_No.max()-test.Day_No.min()+1
temp['Day_No']=np.arange(test.Day_No.min(),test.Day_No.max()+1)

vals = list(train.Course_ID.unique())
i=0
for val in vals:
    temp[str(val)]=np.nan
    
    df_temp=train_[train_['Course_ID']==val]
    df_temp['ds'] = pd.date_range(start='1/1/2018', periods=len(df_temp))
    df_temp.rename(columns={'Sales': 'y'},inplace=True)

    df_temp=df_temp[["ds",'y']]
    df_temp['y'] = np.where(df_temp['y']==0,1,df_temp['y'])
    df_temp['ds'] = pd.to_datetime(df_temp['ds'])

    df_temp_prophet = fbprophet.Prophet(    daily_seasonality=False,
                                        weekly_seasonality=True,
                                        yearly_seasonality=False,
    #                                     seasonality_mode="multiplicative",
                                        interval_width = 0.95,
                                        changepoint_prior_scale=0.05
                                  )
    df_temp_prophet.fit(df_temp)
    df_temp_forecast = df_temp_prophet.make_future_dataframe(periods=n, freq='D')
    
    pred = df_temp_prophet.predict(df_temp_forecast)['yhat'].values
#     pred = df_temp_prophet.predict(df_temp_forecast)
#     pred = pred[['yhat_lower','yhat_upper','yhat']]

    temp[str(val)]=pred[-n:]
    i+=1
#     print(i)
    
    
    

t = temp.melt(id_vars=["Day_No"], 
              var_name="Course_ID",\
              value_name="sales_id_fbprophet")
t['Day_No'] = t['Day_No'] .astype(int)
t['Course_ID'] = t['Course_ID'].astype(int)
t.head()

train = pd.merge(train,t,on=['Course_ID','Day_No'],how='left')  
test = pd.merge(test,t,on=['Course_ID','Day_No'],how='left')  

In [7]:
test.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,sales_id_fbprophet
0,883,883,1,Development,Course,1,0,1,0.007,98.591455
1,884,884,1,Development,Course,1,0,1,0.007,105.764139
2,885,885,1,Development,Course,1,0,1,0.007,88.651621
3,886,886,1,Development,Course,1,0,1,0.007,84.927742
4,887,887,1,Development,Course,0,0,1,0.007,53.767935
